In [25]:
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
import requests

import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

In [26]:

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [27]:
url = 'https://mars.nasa.gov/news/'

browser.visit(url)

html = browser.html

soup = BeautifulSoup(html, 'html.parser')

In [28]:
title = soup.find_all(class_="content_title")
title_info = soup.find_all(class_="article_teaser_body")

In [29]:

news_title = title[1].find('a').text
news_p = title_info[0].text

print(news_title)
print("--------------------------------------------------------------------")
print(news_p)

NASA's Perseverance Rover Is Midway to Mars 
--------------------------------------------------------------------
Sometimes half measures can be a good thing – especially on a journey this long. The agency's latest rover only has about 146 million miles left to reach its destination.


In [30]:
# news_title = soup.find_all('div', class_='content_title')[1].text
# news_p = soup.find_all('div', class_='article_teaser_body')[0].text


print(news_title)
print("--------------------------------------------------------------------")
print(news_p)

NASA's Perseverance Rover Is Midway to Mars 
--------------------------------------------------------------------
Sometimes half measures can be a good thing – especially on a journey this long. The agency's latest rover only has about 146 million miles left to reach its destination.


In [31]:
jet_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

browser.visit(jet_url)

html = browser.html

img_soup = BeautifulSoup(html, 'html.parser')

# img = img_soup.find_all('img')[3]["src"]
# img= img_soup.find_all("img")[3]['src']
# featured_image_url = jet_url + img
# print(featured_image_url)

In [32]:
img_url = img_soup.find('li', class_='slide').a['data-fancybox-href']
featured_image_url = f'https://www.jpl.nasa.gov{img_url}'
print(featured_image_url)


https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA24182_hires.jpg


In [33]:
url_facts = 'https://space-facts.com/mars/'

tables = pd.read_html(url_facts)
len(tables)

3

In [34]:
# print(type(tables))
# print(type(tables[0]))

In [35]:
df = tables[0]
df


,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [36]:
df.columns =['Description', 'Mars'] 

In [37]:
df.set_index('Description', inplace=True)
df.head()

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"


In [38]:
df.to_html('Mars_facts.html')

In [39]:
mars_st = df.to_html(header=True, index=True)
print(mars_st)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mars</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [43]:
geology_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(geology_url)

html = browser.html

geology_soup = BeautifulSoup(html, "html.parser")

main_geology = "https://astrogeology.usgs.gov"


class_url = geology_soup.find_all("div", class_="item")

# Create empty list for each Hemisphere URL.
hemisphere_url = []

for sphere in class_url:
    sphere_url = sphere.find('a')['href']
    hemisphere_url.append(sphere_url)
    
    
print(hemisphere_url)

['/search/map/Mars/Viking/cerberus_enhanced', '/search/map/Mars/Viking/schiaparelli_enhanced', '/search/map/Mars/Viking/syrtis_major_enhanced', '/search/map/Mars/Viking/valles_marineris_enhanced']


In [46]:
hemisphere_image_urls = []
for img in hemisphere_url:
    img_url = main_geology + img
    
    browser.visit(img_url)
    
    html = browser.html

    sphere_soup = BeautifulSoup(html, "html.parser")

    img_title = sphere_soup.find("h2", class_="title").text
    
    title= img_title.split(' Enhanced')[0]
      
    img_url = sphere_soup.find("li").a['href']
    
    hemisphere_image_urls.append({'title': title, 'img_url': img_url})
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
